In [1]:
using BenchmarkTools
using StellarChem
using StellarConstants
using StellarEOS
using StellarOpacity
using StellarEvolution

In [2]:
nvars = 6
nspecies = 2
varnames = [:lnP,:lnT,:lnr,:lum,:H1, :He4]
structure_equations=[StellarEvolution.equationHSE, StellarEvolution.equationT,
                        StellarEvolution.equationContinuity, StellarEvolution.equationLuminosity,
                        StellarEvolution.equationH1, StellarEvolution.equationHe4]
nz = 1000
eos = StellarEOS.IdealEOS(false)
opacity = StellarOpacity.SimpleElectronScatteringOpacity()
sm = StellarModel(varnames, structure_equations, nvars, nspecies, nz, eos, opacity);

In [3]:
StellarEvolution.n1_polytrope_initial_condition(sm, MSUN, 100*RSUN; initial_dt=10*SECYEAR)

StellarEvolution.set_end_step_info(sm)
StellarEvolution.cycle_step_info(sm)
StellarEvolution.set_start_step_info(sm)

StellarEvolution.eval_jacobian!(sm)
StellarEvolution.eval_eqs!(sm)

In [4]:
using LinearSolve
@benchmark begin
    $sm.linear_solver.A = $sm.jac
    $sm.linear_solver.b = $sm.eqs
    corr =solve($sm.linear_solver)
end

BenchmarkTools.Trial: 815 samples with 1 evaluation.
 Range (min … max):  5.321 ms …   8.600 ms  ┊ GC (min … max): 0.00% … 21.92%
 Time  (median):     5.880 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.122 ms ± 593.508 μs  ┊ GC (mean ± σ):  3.65% ±  6.30%

          ▃█▆▅█▃▂ ▄                                            
  ▂▂▂▄▃▄▅██████████▇█▅▅▆▄▅▄▃▃▃▂▃▂▂▁▂▁▂▁▁▂▂▃▄▃▃▅▄▄▄▅▅▄▅▄▃▃▄▃▃▃ ▄
  5.32 ms         Histogram: frequency by time         7.6 ms <

 Memory estimate: 8.73 MiB, allocs estimate: 41.

In [5]:
@benchmark StellarEvolution.eval_jacobian_row!(sm,1)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.249 μs … 793.082 μs  ┊ GC (min … max):  0.00% … 94.55%
 Time  (median):      8.980 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   10.917 μs ±  27.959 μs  ┊ GC (mean ± σ):  10.14% ±  3.92%

   ▃▆██▇▅▃▂▁▁ ▁▁▁▁▁     ▁▁▂▁▁▁▁▁                               ▂
  ▅█████████████████▇████████████▇██▇███████████▇█▇▆▆▆▇▆▆▅▆▆▅▅ █
  8.25 μs       Histogram: log(frequency) by time      17.9 μs <

 Memory estimate: 25.28 KiB, allocs estimate: 211.

In [6]:
@benchmark StellarEvolution.eval_jacobian!(sm)

In [9]:
StellarEvolution.set_options!(sm.opt, "example_options.toml")
rm(sm.opt.io.hdf5_history_filename, force=true)
rm(sm.opt.io.hdf5_profile_filename, force=true)
StellarEvolution.n1_polytrope_initial_condition(sm, MSUN, 100*RSUN, initial_dt=10*SECYEAR)

StellarEvolution.do_evolution_loop(sm)

(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (50, 3.0, 4392.773678888243, 1.8479847518548578e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (100, 3.0, 4935.13025378168, 2.354658929466138e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (150, 3.0, 10104.68689979219, 2.3077459357511185e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (200, 3.0000000000000004, 12245.175949058055, 2.277531470473329e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (250, 2.9999999999999996, 11136.940516364819, 2.249268656630319e7)


...

(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (800, 3.0, 3.518432362278031e6, 6.3630602208055675e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (850, 3.0, 2.7568925239477404e6, 6.378505015328043e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (900, 3.0, 1.785945247759834e6, 6.394438781217843e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (950, 3.0, 3.1929218974736664e6, 6.409527630951707e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (1000, 3.0, 3.0379924689693414e6, 6.424382628001423e7)
Failed to converge, retry


In [10]:
# Compare ρ-T profile with different polytropes

profile_names = StellarEvolution.get_profile_names_from_hdf5("profiles.hdf5")

using CairoMakie, LaTeXStrings

f = Figure()
ax = Axis(f[1,1], xlabel=L"\log_{10}(\mathrm{\rho/[g\;cm^{-3}]})", ylabel=L"\log_{10}(\mathrm{dyne]})")

pname = Observable(profile_names[1])

profile = @lift(StellarEvolution.get_profile_from_hdf5("profiles.hdf5", $pname))
log10_ρ = @lift($profile[!,"log10_ρ"])
log10_P = @lift($profile[!,"log10_P"])

profile_line = lines!(ax, log10_ρ, log10_P, label="real profile")
#compare profile against some polytropes
xvals = LinRange(-13,4,100)
lines!(ax,xvals,(1+1/1) .* xvals .+ 20, label="n=1")
lines!(ax,xvals,(1+1/(1.5)) .* xvals .+ 15, label="n=1.5")
lines!(ax,xvals,(1+1/3) .* xvals .+ 15, label="n=3")
axislegend(ax, position=:rb)

record(f, "rho_P_evolution.mp4", profile_names[1:end];
        framerate = 60) do profile_name
    pname[] = profile_name
end

f

ErrorException: unable to determine if profiles.hdf5 is accessible in the HDF5 format (file may not exist)

In [11]:
# Check evolution of abundance

profile_names = StellarEvolution.get_profile_names_from_hdf5("profiles.hdf5")

using CairoMakie

f = Figure()
ax = Axis(f[1,1], xlabel=L"\mathrm{Mass}\;[M_\odot]", ylabel=L"X")

pname = Observable(profile_names[1])

profile = @lift(StellarEvolution.get_profile_from_hdf5("profiles.hdf5", $pname))
mass = @lift($profile[!,"mass"])
X = @lift($profile[!,"X"])

profile_line = lines!(ax, mass, X, label="real profile")

record(f, "X_evolution.mp4", profile_names[1:end];
        framerate = 60) do profile_name
    pname[] = profile_name
end

f

ErrorException: unable to determine if profiles.hdf5 is accessible in the HDF5 format (file may not exist)

In [12]:
f = Figure()
ax = Axis(f[1,1], xlabel=L"\log_{10}(T_\mathrm{eff}/[K])", ylabel=L"\log_{10}(L/L_\odot)")
xvals = collect(LinRange(0,1,1000))
history = StellarEvolution.get_history_dataframe_from_hdf5("history.hdf5")
lines!(ax, log10.(history[!,"T_surf"]), log10.(history[!,"L_surf"]))
f

UndefVarError: UndefVarError: `Figure` not defined